# Code

In [ ]:
!pip  install  tiktoken==0.4.0  langchain==0.0.231 openai==0.27.8 faiss-cpu==1.7.4 gspread oauth2client nltk pydantic==1.10.8

In [ ]:
import gdown
from langchain.llms import OpenAI
from langchain.docstore.document import Document
import requests
#database
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain.document_loaders import TextLoader

from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile
import ipywidgets as widgets
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re
import getpass
import os
import openai
import tiktoken


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [ ]:
#Work DataBase
from datetime import datetime
import pytz

def create_FAISSdb(source_chunks):
  return FAISS.from_documents(source_chunks, OpenAIEmbeddings())

def save_FAISSdb(db):
  db.save_local("/content/drive/My Drive/faiss_vector_db_" + datetime.now().strftime("%m-%d-%Y-%H-%M-%S")) #База будет сохранена к вам на Google Drive
  return True

def load_FAISSdb(db_name):
  return FAISS.load_local(db_name, OpenAIEmbeddings())

def merg_FAISSdb():
  return True

def delete_FAISSdb():
  return True

In [ ]:
MODEL_TURBO_16K = "gpt-3.5-turbo-16k"
MODEL_TURBO_0613 = "gpt-3.5-turbo-0613"


def num_tokens_from_messages(messages, model=MODEL_TURBO_16K):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo-0301":  # note: future models may deviate from this
        num_tokens = 0
        for message in messages:
            num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == "name":  # if there's a name, the role is omitted
                    num_tokens += -1  # role is always required and always 1 token
        num_tokens += 2  # every reply is primed with <im_start>assistant
        return num_tokens
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.""")

def insert_newlines(textstr: str, max_len: int = 170) -> str:
    words = textstr.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)

def answer_index(system, topic, search_index, temp = 1, verbose = 0, top_similar_documents = 5):

    #Выборка документов по схожести с вопросом
    docs = search_index.similarity_search(topic, k=top_similar_documents)
    if (verbose): print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if (verbose): print('message_content :\n ======================================== \n', message_content)

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"Документ с информацией для ответа пользователю: {message_content}\n\nВопрос клиента: \n{topic}"}
    ]

    print(messages)

    # example token count from the function defined above
    if (verbose): print('\n ===========================================: ')
    if (verbose): print(f"{num_tokens_from_messages(messages, 'gpt-3.5-turbo-0301')} токенов использовано на вопрос")

    completion = openai.ChatCompletion.create(
    model=MODEL_TURBO_16K,
    messages=messages,
    temperature=temp
    )
    answer = insert_newlines(completion.choices[0].message.content)
    return answer  # возвращает ответ

In [ ]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [ ]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

def load_search_indexes(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    #text = response.content.decode('unicode_escape')
    text = response.text
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    md_header_splits = markdown_splitter.split_text(text)

    vectordateBase = FAISS.from_documents(md_header_splits, OpenAIEmbeddings())
    save_FAISSdb(vectordateBase)
    return vectordateBase


In [ ]:

db = load_search_indexes('https://docs.google.com/document/d/1koct2pxGaItFv0l4c4sk7X3xAflWz7pwCFp6WlIkXdw')

#db = load_FAISSdb('/content/drive/My Drive/faiss_vector_db_08-30-2023-13-19-16')

In [ ]:
def answer_user_question_dialog(system: str, db: str, user_question: str, question_history: list) -> str:

    # Если в истории более одного вопроса, применяем суммаризацию
   # summarized_history = ""
   # if len(question_history) > 0:
    #    summarized_history = "Вот краткий обзор предыдущего диалога: " + summarize_questions([q + ' ' + (a if a is not None else '') for q, a in question_history])

    # Добавляем явное разделение между историей диалога и текущим вопросом
   # input_text =summarized_history + "\n\nТекущий ответ пользователя: " + user_question

    # Извлечение наиболее похожих отрезков текста из базы знаний и получение ответа модели
   # answer_text = answer_index(system, input_text, db, temperature, verbose)
    answer_text = answer_index(system, user_question, db, temperature, verbose)

    # Добавляем вопрос пользователя и ответ системы в историю
   # question_history.append(("\n Пользователь: " + user_question, "\n Консультант: " + answer_text if answer_text is not None else ''))

    # Выводим суммаризированный текст, который видит модель
    #if summarized_history != "":
    #if 1 == 0 :
     #   print('****************************')
      #  print(insert_newlines(summarized_history))
       # print('****************************')

    return insert_newlines(answer_text)

In [ ]:
def run_dialog(system_doc_url, knowledge_base_url):
    question_history = []
    dialog = ""
    while True:
        user_question = input('Клиент: ')
        if ((user_question.lower() == 'stop') or (user_question.lower() == 'стоп')):
            break
        answer = answer_user_question_dialog(system_doc_url, knowledge_base_url, user_question, question_history)
        dialog += f'\nПользователь: {user_question} \n Менеджер: {answer}'
        print('\nМенеджер: ', answer+'\n')

    return

In [ ]:
# Промпт моделей машин
promt_url = load_document_text ('https://docs.google.com/document/d/1tjjJQ0s0qKSdmg2Yd69NXX4_RGWLr0up')

NameError: ignored

In [ ]:
temperature=0.3
verbose=0

run_dialog(promt_url, db)